<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/project_configuration/queue_management.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Queue Management

* The queue is used to task labelers with specific assets
* We can do any of the following:
    * Set quality settings
    * Set the order of items in the queue
    * Set the percent of assets to review

In [ ]:
!pip install labelbox -q
!pip install numpy

In [ ]:
import labelbox as lb
from labelbox.schema.quality_mode import QualityMode
import numpy as np
from io import BytesIO
import os
import uuid

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Set up demo project

In [ ]:
## Create two Labelbox  projects
# Project defaults to batch mode with benchmark quality settings if queue mode argument is not provided
# Note that queue mode will be deprecated once dataset mode is deprecated 

project = client.create_project(name="batch-test-project",
                                description="a description",
                                quality_mode=QualityMode.Benchmark, # For Consensus projects use quality_mode = QualityMode.Consensus
                                media_type=lb.MediaType.Image,
                                )

dataset = client.create_dataset(name="queue_dataset")

# Add data to your dataset

In [ ]:
## Example image
uploads = []
global_keys = ["ID-1", "ID-2", "ID-3", "ID-4"]
# Generate data rows
for i in range(1, 5):
  row = {
      "row_data":  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
      "global_key": global_keys[i-1]
    }
  uploads.append(row)


data_rows = dataset.create_data_rows(uploads)
data_rows.wait_till_done()
print("Errors" , data_rows.errors)
print("Dataset status: ", data_rows.status)

# Attach data to your project and set data row priority

In [ ]:
######## Create batches 

# Create the batch 

batch = project.create_batch(
  "batch-demo", # Each batch in a project must have a unique name
  global_keys = global_keys[0:2], # A list of data rows, data row ids or global keys
  priority=5 # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)

batch2 = project.create_batch(
  "batch-demo-2", # Each batch in a project must have a unique name
  #Provide a slice of the data since you can't import assets with global keys that already exist in the project.
  global_keys=global_keys[2:4], # A list of data rows, data row ids or global keys
  priority=1 # priority between 1(Highest) - 5(lowest) 5 is the max priority that can be set
)


print("Batch: ", batch)
print("Batch2: ", batch2)

In [ ]:
print("View the results here:",
      f"https://app.labelbox.com/projects/{project.uid}")
# Click `start labeling` to see the images in order

## Queue Order
- Add priority for each data row
- Remove all the batch priority in your project

In [ ]:
## See current LPOs
print(project.labeling_parameter_overrides().get_many(4))


## Select data rows from batches 
data_rows = []
for b in list(project.batches()):
  for dr in b.export_data_rows(): 
    data_rows.append(dr)
  
## Add LPOs
lpos1 = []
p=1;
for dr in data_rows: 
  lpos1.append((dr, p, 1))
  p+=1


project.set_labeling_parameter_overrides(lpos1)
# Get the project's LPOs

In [ ]:
## Verify LPOs
project_lpos = list(project.labeling_parameter_overrides())
## Remove LPOs
# project.unset_labeling_parameter_overrides(dataset.export_data_rows())

for lpo in project_lpos:
  print(lpo)
  print("Data row:", lpo.data_row().uid)

# Cleanup

In [ ]:
# project.delete()
# dataset.delete()